In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nibabel as nib
from tqdm import tqdm
import time
from tfrecord_utils import *
import tensorflow as tf
from tensorflow.python.ops import array_ops, math_ops, init_ops
from tensorflow.python.framework import dtypes
tf.enable_eager_execution()

/home-local/remedis/Programs/anaconda3/envs/sam/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
TF_RECORD_FILENAME = "/home-local/remedis/data/subdural_lesion/dataset.tfrecords"
# model
inputs = tf.keras.layers.Input(shape=(None, None, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
x = tf.keras.layers.MaxPooling2D(2, 2)(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D(2, 2)(x)
x = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D(2, 2)(x)
#x = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(x)
#x = tf.keras.layers.MaxPooling2D(2, 2)(x)
#x = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = tf.keras.layers.GlobalMaxPooling2D()(x)
#x = tf.keras.layers.Dense(32, activation='relu')(x)
#x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

num_elements = 62
instance_size = (128, 128)

dataset = tf.data.TFRecordDataset(TF_RECORD_FILENAME)\
    .map(lambda record : parse_bag(record, instance_size, 1))\
    .shuffle(buffer_size=num_elements)
    

def running_average(old_average, cur_val, n):
    return old_average * (n-1)/n + cur_val / n

# training
N_EPOCHS = 500
progbar = tf.keras.utils.Progbar(target=num_elements)
opt = tf.train.AdamOptimizer(learning_rate=1e-3)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
batch_size = 32
grads = [tf.zeros_like(l) for l in model.trainable_variables]
for cur_epoch in range(N_EPOCHS):
    print("\nEpoch {}/{}".format(cur_epoch + 1, N_EPOCHS))

    for i, (x, y) in enumerate(dataset):
        with tf.GradientTape() as tape:
            
            repeated_y = np.repeat(y.numpy(), len(x), axis=0)
            
            logits = model(x, training=True)
            
            losses = tf.losses.sigmoid_cross_entropy(
                        multi_class_labels=tf.reshape(repeated_y, repeated_y.shape + (1,)),
                        logits=logits,
                        reduction=tf.losses.Reduction.NONE
            )

            if y.numpy() == 0:
                loss = tf.reduce_mean(losses, axis=0)
                print("True: {:.0f} \tPred: {:.2f}\tLoss: {:.4f}".format(
                    y[0], 
                    tf.reduce_max(logits).numpy(), # prediction is actually the max of the logits for negative case
                    loss.numpy()[0],
                 ))
            else:
                top_polluted_idx = tf.argmin(losses, dimension=1).numpy()[0]
                loss = tf.reduce_min(losses[top_polluted_idx], axis=0)
                
                print("True: {:.0f} \tPred: {:.2f}\tLoss: {:.4f}".format(
                    y[0], 
                    logits[top_polluted_idx].numpy()[0], 
                    loss.numpy(),
                 ))
            
                     
            loss = tf.reshape(loss, (1, 1))

            t = tape.gradient(loss, model.trainable_variables)
            
            #if tf.reduce_sum(t[0]).numpy() == 0 and tf.reduce_sum(t[1]).numpy() == 0:
            #    print("no grad")
            
            for j in range(len(t)):
                grads[j] = running_average(grads[j], t[j], i + 1)
                

            #print(top_polluted_loss.numpy())

        if i > 0 and i % batch_size == 0 or i == num_elements - 1:
            opt.apply_gradients(zip(grads, model.trainable_variables))
            # wipe tape and records
            grads = [tf.zeros_like(l) for l in model.trainable_variables]
            #del tape
            


Epoch 1/500
Instructions for updating:
Use tf.cast instead.
True: 0 	Pred: 1.00	Loss: 1.3047
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 1.00	Loss: 1.2705
True: 0 	Pred: 1.00	Loss: 1.2953
True: 0 	Pred: 1.00	Loss: 1.3037
True: 0 	Pred: 1.00	Loss: 1.3058
True: 0 	Pred: 1.00	Loss: 1.3040
True: 1 	Pred: 1.00	Loss: 0.3133
True: 1 	Pred: 0.61	Loss: 0.4350
True: 1 	Pred: 0.99	Loss: 0.3147
True: 0 	Pred: 1.00	Loss: 1.3044
True: 0 	Pred: 1.00	Loss: 1.3003
True: 0 	Pred: 1.00	Loss: 1.2696
True: 0 	Pred: 1.00	Loss: 1.3083
True: 1 	Pred: 1.00	Loss: 0.3133
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 1.00	Loss: 1.2533
True: 1 	Pred: 1.00	Loss: 0.3133
True: 1 	Pred: 0.93	Loss: 0.3325
True: 0 	Pred: 1.00	Loss: 1.2953
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 1.00	Loss: 1.2912
True: 0 	Pred: 1.00	Loss: 1.3116
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 1.00	Loss: 1.3056
True: 1 	Pred: 1.00	Loss: 0.3133
Tru

True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.47	Loss: 0.7084
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3145
True: 0 	Pred: 0.42	Loss: 0.6966
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.63	Loss: 0.7026
True: 0 	Pred: 0.35	Loss: 0.6938
True: 0 	Pred: 0.32	Loss: 0.6945
True: 0 	Pred: 0.22	Loss: 0.6938
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.42	Loss: 0.6955
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.50	Loss: 0.6961

Epoch 5/500
True: 0 	Pred: 0.47	Loss: 0.6969
True: 0 	Pred: 0.28	Loss: 0.6937
True: 0 	Pred: 0.51	Loss: 0.6956
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.48	Loss: 0.6944
True: 0 	Pred: 0.23	Loss: 0.6941
True: 0 	Pred: 0.41	Loss: 0.6967
True: 0 	Pred: 0.47	Loss: 0.6944
True: 0 	Pred: 0.40	Loss: 0.6947
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.50	Loss: 0.69

True: 0 	Pred: 0.81	Loss: 0.6966
True: 0 	Pred: 0.47	Loss: 0.6955
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.52	Loss: 0.6968
True: 0 	Pred: 0.51	Loss: 0.6978
True: 0 	Pred: 0.41	Loss: 0.6943
True: 0 	Pred: 1.00	Loss: 0.7073
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.47	Loss: 0.6958
True: 0 	Pred: 0.12	Loss: 0.6933
True: 0 	Pred: 0.52	Loss: 0.6973
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.46	Loss: 0.6963
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.83	Loss: 0.6968

Epoch 9/500
True: 0 	Pred: 0.09	Loss: 0.6935
True: 0 	Pred: 0.47	Loss: 0.7033
True: 0 	Pred: 0.45	Loss: 0.6964
True: 0 	Pred: 0.45	Loss: 0.6945
True: 0 	Pred: 0.45	Loss: 0.7047
True: 0 	Pred: 0.26	Loss: 0.6938
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.47	Loss: 0.7001
True: 0 	Pred: 0.46	Loss: 0.6973
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.69

True: 0 	Pred: 0.42	Loss: 0.6950
True: 0 	Pred: 0.36	Loss: 0.6977
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.21	Loss: 0.6940
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.29	Loss: 0.6952
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.32	Loss: 0.6950
True: 0 	Pred: 0.15	Loss: 0.6934
True: 0 	Pred: 0.36	Loss: 0.6955
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.24	Loss: 0.6940
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.01	Loss: 0.6932
True: 0 	Pred: 0.39	Loss: 0.7001
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.10	Loss: 0.6933
True: 0 	Pred: 0.59	Loss: 0.6945

Epoch 13/500
True: 0 	Pred: 0.30	Loss: 0.6942
True: 0 	Pred: 0.28	Loss: 0.6950
True: 0 	Pred: 0.41	Loss: 0.6949
True: 0 	Pred: 0.31	Loss: 0.6949
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.38	Loss: 0.6953
True: 0 	Pred: 0.18	Loss: 0.6936
True: 0 	Pred: 0.40	Loss: 0.6940
True: 1 	Pred: 0.00	Loss: 0.6

True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.01	Loss: 0.6932
True: 0 	Pred: 0.30	Loss: 0.6948
True: 0 	Pred: 0.10	Loss: 0.6934
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.20	Loss: 0.6942
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.43	Loss: 0.6940
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.33	Loss: 0.6943
True: 0 	Pred: 0.35	Loss: 0.6966
True: 0 	Pred: 0.29	Loss: 0.6953
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.04	Loss: 0.6932
True: 0 	Pred: 0.34	Loss: 0.6942
True: 0 	Pred: 0.33	Loss: 0.6938
True: 0 	Pred: 0.35	Loss: 0.6940

Epoch 17/500
True: 0 	Pred: 0.22	Loss: 0.6935
True: 0 	Pred: 0.27	Loss: 0.6945
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.98	Loss: 0.3198
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.19	Loss: 0.6941
True: 0 	Pred: 0.29	Loss: 0.6944
True: 0 	Pred: 0.67	Loss: 0.6947
True: 1 	Pred: 0.00	Loss: 0.6

...

True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	P

True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	P

True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	P

True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 1.00	Loss: 0.3133
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	P

True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 1 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	Pred: 0.00	Loss: 0.6931
True: 0 	P

In [ ]:

'''
# This doesn't work for some reason even though I think it should.
batch_size = 16
grads = [tf.zeros_like(l) for l in model.trainable_variables]
for cur_epoch in range(N_EPOCHS):
    print("\nEpoch {}/{}".format(cur_epoch + 1, N_EPOCHS))
    #progbar.update(0)
    
    for i, (x, y) in enumerate(dataset):
        with tf.GradientTape() as tape:
            
            logits = model(x, training=True)

            if y.numpy() == 1:
                target_pred = tf.reduce_min(logits, axis=0)
            else:
                target_pred = tf.reduce_mean(logits, axis=0)

            loss = tf.losses.sigmoid_cross_entropy(y, target_pred, reduction=tf.losses.Reduction.NONE)
            t = tape.gradient(loss, model.trainable_variables)
            
            if tf.reduce_sum(t[0]).numpy() == 0 and tf.reduce_sum(t[1]).numpy() == 0:
                print("no grad")

            for j in range(len(t)):
                grads[j] = running_average(grads[j], t[j], i + 1)

            print("True: {:.0f} Pred: {:.0f} Loss: {:.4f}".format(
                    y[0], 
                    np.round(target_pred[0]), 
                    loss.numpy()[0],
                 ))

        if i > 0 and i % batch_size == 0 or i == num_elements - 1:
            #print("applied gradient")
            opt.apply_gradients(zip(grads, model.trainable_variables))
            # wipe records
            grads = [tf.zeros_like(l) for l in model.trainable_variables]
            #del tape

        #progbar.add(1, values=[("loss", loss)])'''

In [4]:
# prediction
correct_count = 0
total_count = 0
for i, (x, y) in enumerate(dataset):
    
    pred = model(x)
    pred = tf.reduce_max(pred)
    total_count += 1
    if np.round(pred.numpy()) == y.numpy()[0]:
        correct_count += 1
print("{:.2%}".format(correct_count/total_count))

69.35%


In [2]:
import numpy as np

In [13]:
logits = np.zeros((100,1,5))

In [14]:
logits.shape

(100, 1, 5)

In [16]:
logits[0] = np.array([[1, 0, 0, 0, 0]])

In [17]:
logits[1] = np.array([[0, 1, 1, 0, 0]])

In [24]:
logits[30] = np.array([[0, 1, 0, 0, 2]])

In [25]:
np.argmax(logits, axis=0)

array([[ 0,  1,  1,  0, 30]])